# ig_analysis

### Read in the datasets created

In [ ]:
from lib.utils import *

followers_df, following_df, feed_df = get_user_dataframes()

### How many followers does the user have?

In [ ]:
print_num_of_followers(followers_df)

### How many accounts don't follow-back the user?

In [ ]:
print_no_follow_back_accounts(followers_df, following_df)

### What is the most liked post?

In [ ]:
render_top_n_items_summary(feed_df, 'likes_count', 1)

### What is the least liked post?

In [ ]:
render_top_n_items_summary(feed_df, 'likes_count', 1, True)

### What is the like count average across all posts?

In [ ]:
print_like_count_average(feed_df)

### What are the top-5 posts based on like count?

In [ ]:
render_top_n_items_summary(feed_df, 'likes_count', 5)

### What days, times, hashtags, and words were used in the top-5 posts based on like count?

In [ ]:
render_top_n_items_statistics(feed_df, 'likes_count', 5)

### TODO: What are the top-5 users who like the user's posts?
### TODO: What is the post with most comments (how many comments)?
### TODO: What is the post with least comments (how many comments)?
### TODO: What is the comment count average across all posts?
### TODO: What are the top-5 posts based on comment count?
### TODO: What days, times, hash-tags, and words were used in the top-5 posts based on comment count?
### TODO: What are the top-5 users who comment on the user's posts?